In [1]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from bson.son import SON
from bson.code import Code
import json
import pandas as pd
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
# These are packages need for natural language processing:
import nltk
from __future__ import division, unicode_literals 
from textblob import TextBlob as tb
import math
from datetime import datetime
from dateutil import parser

In [2]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import amazon_data_sort 
from amazon_data_sort import get_b_product, get_reviews
import doc_summarization as ds

In [3]:
#For Topic Modelling LDA:
import lda
import scipy
import gensim

from gensim import corpora, models, similarities
from gensim.models import hdpmodel, ldamodel
from itertools import izip

/usr/local/lib/python2.7/site-packages/numpy/lib/utils.py:95: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)
/usr/local/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/local/lib/python2.7/

In [4]:
client = MongoClient()
mbf = client.dsbc.mbf
product_info = client.dsbc.product_info
name = product_info.find({}, {"_id":0,"asin":1,"title":1,"price":1})
beauty_list  = {}
for id_num in name:
    try:
        beauty_list[id_num["asin"]]=[id_num["title"],id_num["price"]]
    except:
        pass
#Creating database for reviews beauty file
rbf = client.dsbc.rbf

In [5]:
import reviews as r
import product_user_profile as pup

In [6]:
#These codes help find all sentence with the word of interests
#It's help full to see the exact condition surrounding the word
#to built all the terms for a category
from nltk import tokenize
for index in xrange(10):
    a = r.reviews(beauty_list,index)['Reviews']
    for i in a:
        sentences  = tokenize.sent_tokenize(i[0])
        for sentence in sentences:
            if "sun" in sentence:
                print sentence
                print ""
                print ""

As much as any of us hate to admit, how well we age is determined predominately by genetics plus exercise, good nutrition, hydration, sunscreen and regular skin care.


I will tell you, using this and staying out of the sun have protected my facial skin so much that, at age 56, I still have no wrinkles unless I smile really hard and you look up close.


"If you use this cream every day and stay out of the sun faithfully and/or use sunscreen, you will have beautiful skin long after your friends have started to get very noticeable lines.Regards,Karen


One of the few I've tried that doesn't smell like it contains sunscreen.


It's tint-free, which I also like.I use it primarily on my face and lower arms where the sun hits.




In [7]:
print r.reviews(beauty_list,5)
pup.product_profiler(r.reviews(beauty_list,5))

{u'B00BJEJY9E': [u'SK II Men UV Protect Moisturizer SPF 30 PA +++ New 50g / 1.7oz', 98.11], 'Reviews': [[u'This is the first moisturizer with SPF I have found that I love as a moisturizer. It has a medium texture and leaves no residue on the face. I strongly recommend it.', 5.0]]}


{'acne_score': 0.0,
 'anti_aging_score': 0.0,
 'combination_skin_score': 0.0,
 'dry_skin_score': 0.0,
 'oily_skin_score': 0.0,
 'redness_score': 0.0,
 'sensitive_score': 0.0,
 'sunscreen_score': 0.0}

###Now transfer everything onto MongoDB

In [20]:
item_profile = client.dsbc.item_profile

In [15]:
def summarize_item(item_list,item_index):
    all_item_info = {}
    all_item_info = {item_list.keys()[item_index]:item_list.values()[item_index][0], 
                     "price":item_list.values()[item_index][1]}
    all_item_score = pup.product_profiler(r.reviews(item_list,item_index))
    all_item_info.update(all_item_score)
    return all_item_info

In [ ]:
for i in xrange(5570):
    beauty_item = summarize_item(beauty_list, i)
    item_profile.insert_one(beauty_item)

In [16]:
for i in xrange(5):
    print i

0
1
2
3
4


In [9]:
def skin_type_counter(list_of_text):
    dry_skin = 0
    oily_skin = 0
    combination_skin = 0
    sensitivity = 0
    total_reviews = len(list_of_text)
    for review_text in list_of_text:
        new_text = review_text[0].lower()
        dry_skin += new_text.count("dry skin")
        oily_skin += new_text.count("oily skin")
        combination_skin += new_text.count("combination skin")
        sensitivity += new_text.count("sensitive")*tb(new_text).sentiment.polarity
    return [dry_skin, oily_skin, combination_skin, sensitivity/(total_reviews+1)]
    

In [11]:
a = get_reviews(beauty_list,42)['Reviews']
# for i in a:
#     print i
#     print skin_type_counter([i])
#     print ""
#     print ""

In [15]:
words = []
sentences  = nltk.word_tokenize(a[0][0])

In [28]:
stopwords = nltk.corpus.stopwords.words("english")\
            + ['usps','shipping','$','#', 'very', 'amazon','every',
                '&','order','seller','packaging','manufacture', 'ever',
                'package','cute','delivery', '.',',',"return", "might"
                '?','...','day','week', 'weeks']


In [29]:
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    text = [word for word in tokens if word not in stopwords]
    tagged_text = nltk.pos_tag(text)
    for word, tag in tagged_text:
        words.append({"word": word, "pos": tag})

In [32]:
from nltk.stem.wordnet import WordNetLemmatizer

In [41]:
nouns = []
lem = WordNetLemmatizer()
noun_only = [noun for noun in words if noun["pos"] in ["NN", "NNS"]]
for noun_word in noun_only:
    nouns.append(lem.lemmatize(noun_word["word"]))
    

In [44]:
import logging
import gensim
from gensim import corpora
from gensim.corpora import BleiCorpus